In [1]:
#!pip install lightgbm

In [2]:
#!pip install awswrangler

In [3]:
import boto3
import io
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import awswrangler as wr
#import s3fs
#from fastparquet import write

import joblib

#modeling
import statsmodels.api as sm
from statsmodels.api import add_constant
from sklearn import preprocessing
from sklearn.model_selection import train_test_split,ShuffleSplit, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import roc_curve, auc, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay,\
classification_report, accuracy_score, f1_score, precision_recall_curve
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
import sklearn.metrics as metrics
from sklearn.metrics import make_scorer, roc_auc_score, log_loss, brier_score_loss
from sklearn.utils import resample


import lightgbm as lgbm

plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings("ignore")

In [4]:
#pd.set_option('display.max_rows', 200)

#### Laod Data from S-3

In [5]:
session = boto3.Session()
s3 = session.client('s3')

In [6]:
bucket_name = 'ppsg-collections'
file_name = '2021-06-01/consolidated/random_data/consolidated_sample_expanded_treated.parquet'

In [7]:
%%time
obj = s3.get_object(Bucket = bucket_name, Key = file_name)
df = pd.read_parquet(io.BytesIO(obj['Body'].read())) 

CPU times: user 1min 51s, sys: 4min 59s, total: 6min 51s
Wall time: 44.5 s


In [8]:
#df =  df[df['PastDue'] <=31]

In [9]:
out_date = '2021-02-01'

#### Performance Metrics

In [10]:
#roc curve
def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.show()

In [11]:
def fn_importance_features(model):
    clf = model
    feature_importances_data = []
    features = train_X.columns
    for feature_name, feature_importance in zip(features, clf.feature_importances_):
        feature_importances_data.append({
            'feature': feature_name,
            'importance': feature_importance
        })

    # Plot the results
    pd.DataFrame(feature_importances_data)\
        .set_index('feature')\
        .sort_values(by='importance')[-15::]\
        .plot(title='Top 15 most important features: '+ str(type(model).__name__),
              kind='barh',figsize=(10, 6),
              color='#348ABD',alpha=0.6,
              lw='1', edgecolor='#348ABD',grid=False,)

In [12]:
#key performance metrics
def fn_metrics(act, prob, pred):
    #ROC
    fpr, tpr, thresholds = roc_curve(act, prob)
    roc_auc = auc(fpr, tpr)
    plot_roc_curve(fpr, tpr)
    
    print ('ROC value of:{:.2f}'.format(roc_auc*100))
    print ('Accuracy:{:.2f}'.format(accuracy_score(act, pred)*100))
    print ('F1 score:{:.2f}'.format(f1_score(act, pred)*100))
    print ('Recall:{:.2f}'.format(recall_score(act, pred)*100))
    print ('Precision:{:.2f}'.format(precision_score(act, pred)*100))
    print ('\n clasification report:\n',classification_report(act, pred))
     
    #plot confusion matrix    
    fig, ax = plt.subplots(1,1,figsize=(8,6))
    plt.rcParams.update({'font.size': 16})
    ConfusionMatrixDisplay(confusion_matrix(pred, act, labels=[1,0]), 
                       display_labels=['Yes_Down2','No_Down2']).plot(values_format=".0f",ax=ax)

    ax.set_xlabel("Actual Label")
    ax.set_ylabel("Predicted Label")
    plt.show()

#### Features

In [13]:
features = [
'Down2',
'nextpaymentduedt',
'CampaignProfile',
'rollback',
                        
#origination
'FICOScore',
'FICO_flag',
'FICO_200',
'LTV',
#'currentLTV', 
'FrontDTI', 
'origrate',
'origbal', 
'origterm', 
'origpropvalue',
                       
#balance related
'current_total_pmt',
'currentpipayment',
'priorupb',
'upb',
'upb_change',
'TI_change',   
'PI_change',
'Escrow_change',            
'curtailment_3months',  
'curtailment_6months',
'curtailment_12months',

#income related            
'TI',
'EMPL_Income',
'OTHR_Income',
'PNSN_Income',
'RNTL_Income',
'SEMP_Income',
'SSEC_Income',
'EMPL_Income_per', 
'OTHR_Income_per', 
'PNSN_Income_per', 
'RNTL_Income_per',
'SEMP_Income_per', 
'SSEC_Income_per',
            
#time related 
'age',
'CurrentAmortTerm',
'nextpaymentdue_day',
'nextpayment_weekday',
'days_in_month',            
            
#dynamic variabels
'rundt_day',
'loanstatusid_lag1',
    
    
# flags 
'forebearance_status_flag',
'drafting_ind_flag',
'Covid_flag',
'selfemployed_flag',
'DUtype_flag',
'doctype_flag',
'tract_income_category_flag',
'application_income_category_flag',
'mod_flag',
'draft_flag',
'ACHPmtSet_flag',
'EscrowFlag',
'CRAFlag',
'HomeReadyHomePossibleFlag',
'streamline_flag',
'HighBalanaceFlag',
    
#others 
'producttype',
'priorstatus',
'LoanPurpose',
'borrowercount',
'fsttimehomebuyer',  
'PropState',
#'zipcode',
'CBSACode',
'samemailingandpropertyzip',
'PropertyType',
'Units',
'Occupancy',
'channel',
'Entity',
'PaymentInFullStopCode',
'apprbucket',
'amorttype',
'termbucket',

           
#Right Party Contact 
'rpc_3months',
'rpc_6months',
'rpc_12months',
'rpc_24months',
            
#Broken Promise
'bph_3months',
'bph_6months',
'bph_12months',
'bph_24months',
            
#Reversal related             
'reversalcount_3months',
'reversalcount_6months',
'reversalcount_12months',
'reversalamt_3months',
'reversalamt_6months',
'reversalamt_12months',  
      
# NSF
'nsfcount_3months',
'nsfcount_6months',
'nsfcount_12months',
'nsfamt_3months',
'nsfamt_6months',
'nsfamt_12months',
    
# CA values,
#'leadg_Value', 
'OpenLossDraftClaimAmount',
    
#Delinquency, BK related
'status24months',
#'bk24months',
'monthssincecurrent',
'monthssince30',
'monthssince60',
'monthssince90',
'monthssince120',
'monthssinceFC',
'monthssinceBK',
'prior24historyCurrent',
'prior24history30D',
'prior24history60D',
'prior24history90D',
'prior24history120D',
'prior24historyFC',
'prior12historyCurrent',
'prior12history30D',
'prior12history60D',
'prior12history90D',
'prior12history120D',
'prior12historyFC',
'prior6historyCurrent',
'prior6history30D',
'prior6history60D',
'prior6history90D',
'prior6history120D',
'prior6historyFC',
'Dlq_Count_12M',            

            
#avg payment day
#'AvgPmtDay_3M',
#'AvgPmtDay_6M', 
#'AvgPmtDay_12M'
'AvgPmtDay_3M_bucket',
'AvgPmtDay_6M_bucket',
'AvgPmtDay_12M_bucket',
]

In [14]:
ml_dataset =df[features]
ml_dataset.shape

(28469560, 128)

In [15]:
ml_dataset=ml_dataset.dropna()
ml_dataset.shape

(28207780, 128)

In [16]:
#del df

#### Pre-Processing: Categorical

In [17]:
%%time
cat_features = [
'CampaignProfile',
'AvgPmtDay_3M_bucket',
'AvgPmtDay_6M_bucket',
'AvgPmtDay_12M_bucket',
'rollback',
'rundt_day',    
'FICO_flag',
'FICO_200',    
'status24months',
'loanstatusid_lag1',    
'forebearance_status_flag',
'drafting_ind_flag',
'Covid_flag',
'selfemployed_flag',
'DUtype_flag',
'doctype_flag',
'CRAFlag',
'tract_income_category_flag',
'application_income_category_flag',
'mod_flag',
'draft_flag',
'ACHPmtSet_flag',
'EscrowFlag',
'HomeReadyHomePossibleFlag',
'streamline_flag',
'HighBalanaceFlag', 
'producttype',
'priorstatus',
'LoanPurpose',
'borrowercount',
'fsttimehomebuyer',  
'PropState',
#'zipcode',
'CBSACode',
'PropertyType',
'Units',
'Occupancy',
'channel',
'samemailingandpropertyzip',
'Entity',
'PaymentInFullStopCode',
'apprbucket',
'amorttype',
'termbucket'
]   

for f in ml_dataset[cat_features]:
    print(f)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(ml_dataset[f].values))
    ml_dataset[f] = lbl.transform(list(ml_dataset[f].values))

CampaignProfile
AvgPmtDay_3M_bucket
AvgPmtDay_6M_bucket
AvgPmtDay_12M_bucket
rollback
rundt_day
FICO_flag
FICO_200
status24months
loanstatusid_lag1
forebearance_status_flag
drafting_ind_flag
Covid_flag
selfemployed_flag
DUtype_flag
doctype_flag
CRAFlag
tract_income_category_flag
application_income_category_flag
mod_flag
draft_flag
ACHPmtSet_flag
EscrowFlag
HomeReadyHomePossibleFlag
streamline_flag


KeyboardInterrupt: 

#### Pre-processing: Numerical  (Min Max scaling)

In [18]:
%%time
num_features = [x for x in ml_dataset.columns if x not in [cat_features, 'Down2', 'nextpaymentduedt']]

for f in ml_dataset[num_features] :
    if ml_dataset[f].dtype in ('float', 'int'):
        _min = ml_dataset[f].min()
        _max = ml_dataset[f].max()
        scale = _max - _min
        shift = _min

        ml_dataset[f] = (ml_dataset[f] - shift).astype(np.float64) / scale

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



#### Imbalance Data Treatment (simplified version - oversample)

In [ ]:
# #Resampling: upsample of Fraud count

# # Separate majority and minority classes
# ml_dataset_majority = ml_dataset[ml_dataset.Down2==0]
# ml_dataset_minority = ml_dataset[ml_dataset.Down2==1]

# # Upsample minority class
# ml_dataset_minority_upsampled = resample(ml_dataset_minority,
#                                  replace=True,     # sample with replacement
#                                  n_samples=500000,  #arbitrary number   
#                                  random_state=123) # reproducible results

# # Combine majority class with upsampled minority class
# ml_dataset= pd.concat([ml_dataset_majority, ml_dataset_minority_upsampled])

# # Display new class counts
# ml_dataset.Down2.value_counts()

#### Create a Target

In [ ]:
ml_dataset['__target__'] = ml_dataset['Down2']
del ml_dataset['Down2']

# Remove rows for which the target is unknown.
ml_dataset = ml_dataset[~ml_dataset['__target__'].isnull()]

#### Split Train/Validation/Test

In [ ]:
# Train data < outdate
train = ml_dataset[ml_dataset['nextpaymentduedt'] < out_date]

# split train to train and  test (validation) data
train, valid = train_test_split(train, test_size=0.15, random_state=101)

# test data >= outdate
test = ml_dataset[ml_dataset['nextpaymentduedt'] >= out_date]

train = train.drop(columns=['nextpaymentduedt'])
valid = valid.drop(columns=['nextpaymentduedt'])
test = test.drop(columns=['nextpaymentduedt'])

#train, test = train_test_split(ml_dataset, test_size=0.2, random_state=101)
print ('Train data has %i rows and %i columns' % (train.shape[0], train.shape[1]))
print ('Valiation data has %i rows and %i columns' % (valid.shape[0], valid.shape[1]))
print ('Test data has %i rows and %i columns' % (test.shape[0], test.shape[1]))

In [ ]:
train_X = train.drop('__target__', axis=1)
valid_X = valid.drop('__target__', axis=1)
test_X = test.drop('__target__', axis=1)

train_Y = np.array(train['__target__'])
valid_Y = np.array(valid['__target__'])
test_Y = np.array(test['__target__'])

#### LGBM Model

In [ ]:
%%time
# with validation and early stoppage: ROC is better than recall

import time
import lightgbm as lgb

t = time.time()
folds = 6    #10
param_comb = 6  #2 #5 #10

lgb_clf = lgb.LGBMClassifier(objective = 'binary', boosting_type = 'gbdt',  random_state=101)
#                              early_stopping_rounds =30, 
lgb_parameters  = {
             ''learning_rate': sp_randFloat(),
             'n_estimators':sp_randInt(200, 500), 
             'max_depth':sp_randInt(5, 50), 
             'num_leaves': sp_randInt(10, 100), 
             'min_child_samples': sp_randInt(100, 500), 
             'min_child_weight': [1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             #'subsample': sp_randFloat(), 
             'subsample_for_bin': [50000, 100000, 200000, 300000, 400000, 500000],
             'colsample_bytree': sp_randFloat(),
             'reg_alpha': [0, 1e-1, 10,  20, 50, 100, 200],
             'reg_lambda': [0, 1e-1, 10, 20, 50, 100, 200],
             'scale_pos_weight': [5, 7, 8, 10, 15, 20, 25],
             'min_split_gain': [0.5e-1, 1e-1, 2e-1, 3e-1, 5e-1, 7.5e-1, 1],
             'bagging_fraction': sp_randFloat()

#socring parameters 
scoring = { 'AUC': 'roc_auc',  'Log_loss': 'neg_log_loss', 'F1': 'f1','Recall': 'recall', 'Precision': 'precision'} 

# stratified k-fold
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 101)


# Randomized Grid Search
random_search = RandomizedSearchCV(lgb_clf, param_distributions=lgb_parameters, n_iter=param_comb, 
                                     scoring= scoring,  refit='Recall',   #Recall #AUC #Log_Loss #Brier_loss
                                     #n_jobs=-1,
                                     cv=skf.split(train_X, train_Y), verbose=3, random_state=101)


# fit training data and eval data for early stopping (validate with validation data)
random_search.fit(train_X, train_Y, eval_set = [(valid_X, valid_Y)], eval_metric =  'recall',
                                                early_stopping_rounds = 50, verbose=50)
#eval_metric = 'auc', 'recall', ['recall','binary_logloss']

# results
tune_results = random_search.cv_results_

#best parameter setting
print('Best Parameter setting: ',  random_search.best_params_)
    
#best ROC score
print('Best score Training: ', random_search.best_score_)

print(f'Done, time = {time.time() - t}')

#### Export model, create a pickle object

#### Method 1. Export to Local Python and Load

In [ ]:
#Export to local
joblib.dump(random_search.best_estimator_, 'lgbm_0610_model_replicated.pkl')

#load pickle local python
clf_tuned_lgb =joblib.load('lgbm_0610_model_replicated.pkl')

In [ ]:
#clf_tuned_lgb

#### Method 2. Export to S3 and Load from S3

In [ ]:
# Export to S3
import pickle
pickle_file = 'lgbm_0610_model_replicated.pkl'
file_path= 'Feb21_test/Model/'
key_name = file_path + pickle_file

pickle_byte_obj = pickle.dumps(random_search.best_estimator_) 
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket_name,key_name).put(Body=pickle_byte_obj)

In [ ]:
# load model object from s3
bucket = boto3.resource("s3").Bucket(bucket_name)
with BytesIO() as modelfo:
    bucket.download_fileobj(Key=key_name, Fileobj=modelfo)
    clf_tuned_lgb = joblib.load(modelfo)

#### Model Fits

In [ ]:
fn_importance_features(clf_tuned_lgb)

In [ ]:
#### In-sample metrics
pred_Y_in = clf_tuned_lgb.predict(train_X)
pred_Y_prob_in = clf_tuned_lgb.predict_proba(train_X)[:,1]
tuned_lgb_prob_in = [train_Y, pred_Y_prob_in]
fn_metrics(train_Y, pred_Y_prob_in, pred_Y_in)

In [ ]:
#Out-of-sample metrics 
pred_Y_out = clf_tuned_lgb.predict(test_X)
pred_Y_prob_out = clf_tuned_lgb.predict_proba(test_X)[:,1]
tuned_lgb_prob_out = [test_Y, pred_Y_prob_out]

#print out ROC curve, metrics, and confuction matrix
fn_metrics(test_Y, pred_Y_prob_out, pred_Y_out)

#### In-Sample Results

In [ ]:
## in-sample results 
#df_in_predict = pd.DataFrame({'prob': pred_Y_prob_in, 'predict': pred_Y_in}, columns=['prob', 'predict'], index = train_X.index.copy())

# Build scored dataset
#results_in = df.join(df_in_predict, how='right')

#### Out-of-time Results

In [ ]:
del ml_dataset

In [ ]:
## out-of-sample results 
df_out_predict = pd.DataFrame({'prob': pred_Y_prob_out, 'predict': pred_Y_out}, columns=['prob', 'predict'], index = test_X.index.copy())

# Build scored dataset
results_out = df.join(df_out_predict, how='left')
#results_out = results_out.sort_index()

In [ ]:
#df_in = results_out[results_out['nextpaymentduedt'] < outdate]
df_out = results_out[results_out['nextpaymentduedt'] >= out_date]

In [ ]:
#del train, train_X, valid, valid_X, test, test_X

#### Export Results

In [ ]:
%%time
#import awswrangler as wr
try:
    wr.s3.to_parquet(
        df=df_out,
        path='s3://ppsg-collections/2021-06-01/processing/feb21_out_0610.parquet'
    )
except Exception as e:
    print(str(e))

In [ ]:
# %%time
# ##import awswrangler as wr
# wr.s3.to_parquet(
#     df=results_out,
#     path='s3://ppsg-collections/2021-06-01/processing/results_out_0610.parquet'
# )